In [1]:
import pandas as pd
from pydantic_ai import Agent
import dotenv
from tqdm.auto import tqdm
from whatstk import WhatsAppChat
import uuid


# dotenv.load_dotenv()

# file_path = '../data/WhatsAppChat/_chat.txt'

# wa_chat = WhatsAppChat.from_source(filepath=file_path)

# file_path = '../data/WhatsAppChat/whatsmeow_contacts_202502090741.csv'
# contacts_df = pd.read_csv(file_path)

# def match_and_rename_users(wa_chat: WhatsAppChat, contacts_df: DataFrame) -> WhatsAppChat:
#     dict_of_users = defaultdict(list)

#     contacts_df.fillna("", inplace=True)

#     for index, row in contacts_df.iterrows():
#         phone_number = row['their_jid'].split('@')[0]
#         # Using standard hyphen and handling variable length numbers
#         long_number = f'+{phone_number[0:3]} {phone_number[3:5]}-{phone_number[5:8]}-{phone_number[8:]}'
#         dict_of_users[phone_number].extend([long_number])

#         if row['full_name']:
#             dict_of_users[phone_number].extend([
#                 row['full_name'],
#                 f"~ {row['full_name']}"
#             ])

#         elif row['push_name']:
#             dict_of_users[phone_number].extend([
#                 row['push_name'],
#                 f"~ {row['push_name']}"
#             ])

#     dict_of_users = {k: list(set(v)) for k, v in dict_of_users.items()}

#     swapped_names = wa_chat.rename_users(mapping=dict_of_users)
#     return swapped_names

# # renames users to phone numbers for latter tagging use
# renamed_wa_chat = match_and_rename_users(wa_chat, contacts_df)

# chat_df = renamed_wa_chat.df

# #filter out not interesting messages
# patterns = '|'.join([
# "This message was deleted",
# "you deleted this message.",
# "you deleted this message as admin",
# "Contact card omitted",
# "GIF omitted",
# "image omitted",
# "video omitted",
#  r'\d{3}[-‐]?\d{3,4}\s+left',  # matches phone number patterns followed by "left"
# "requested to join",
# "Your security code with ",
# 'security code',
# 'pinned a message',
# 'changed their phone number to a new number',
# joined using this group\'s invite link',
# 'Waiting for this message'
# ])

# chat_df = chat_df[~chat_df['message'].str.contains(patterns, case=False, na=False, regex=True)]


# chat_df['group'] = "GenAI Israel"
# chat_df['id'] = [f"imported_{uuid.uuid4()}" for _ in range(len(chat_df))]

/Users/ilan/workspace/kelet/wa_llm/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
import pandas as pd

chat_df_file_path = "../data/WhatsAppChat/chat_gen_ai_1_2025_02_17.csv"
# chat_df_file_path = "../data/WhatsAppChat/chat_gen_ai_2_2025_02_17.csv"
chat_df = pd.read_csv(chat_df_file_path)

Groups chat to Conversations. From Conversations to topics.

In [2]:
import logfire
import asyncio
from models import Message
from typing import Dict
from daily_ingest.daily_ingest import get_conversation_topics, load_topics
from sqlalchemy.ext.asyncio import create_async_engine
from sqlalchemy.orm import sessionmaker
from sqlmodel.ext.asyncio.session import AsyncSession
from sqlmodel import select
from config import Settings
from models.group import Group
from voyageai.client_async import AsyncClient
from sqlalchemy.exc import SQLAlchemyError


settings = Settings()  # pyright: ignore [reportCallIssue]

logfire.configure()
async_engine = create_async_engine(
    settings.db_uri,
    pool_size=50,
    max_overflow=400,
    pool_timeout=90,
    pool_pre_ping=True,
    pool_recycle=600,
    future=True,
)
async_session = sessionmaker(
    bind=async_engine,
    autocommit=False,
    autoflush=False,
    expire_on_commit=False,
    class_=AsyncSession,
)
# db_session = AsyncSession(engine)

embedding_client = AsyncClient(
    api_key=settings.voyage_api_key, max_retries=settings.voyage_max_retries
)


async def run_query(query, async_session: AsyncSession):
    async with async_session() as session:
        try:
            result = await session.exec(statement=query)
        except SQLAlchemyError as sql_ex:
            await session.rollback()
            raise sql_ex
        finally:
            await session.commit()

    return result


group_from_file = (
    await run_query(
        (select(Group).where(Group.group_name == chat_df["group"][1])), async_session
    )
).all()
group_jid = group_from_file[0].group_jid

# TODO: add username from chat_df to sender's table. We should upsert and and ignore if already exists.


def _identify_conversations(df, time_threshold_minutes):
    """
    Identifies separate conversations in WhatsApp chat data based on time gaps between messages.

    Parameters:
    df (pandas.DataFrame): DataFrame containing WhatsApp chat data with 'date' column
    time_threshold_minutes (int): Time gap (in minutes) to consider as a new conversation

    Returns:
    pandas.DataFrame: Original DataFrame with an additional 'conversation_id' column
    """
    # Make sure we're working with a copy
    df = df.copy()

    # Convert date column to datetime if it's not already
    df["date"] = pd.to_datetime(df["date"])

    # Sort by date
    df = df.sort_values("date")

    # Initialize conversation ID
    conversation_id = 0
    conversation_ids = [conversation_id]

    # Iterate through messages (except the first one)
    for i in range(1, len(df)):
        current_time = df.iloc[i]["date"]
        previous_time = df.iloc[i - 1]["date"]

        # Calculate time difference in minutes
        time_diff = (current_time - previous_time).total_seconds() / 60

        # If time difference is greater than threshold, start new conversation
        if time_diff > time_threshold_minutes:
            conversation_id += 1

        conversation_ids.append(conversation_id)

    # Add conversation IDs to DataFrame
    df["conversation_id"] = conversation_ids
    return df


def _create_user_mapping(wa_df: pd.DataFrame) -> Dict[str, str]:
    """
    Creates a mapping of usernames to shortened names (@user_[id]),
    where more frequent speakers get lower IDs.

    Parameters:
    df (pandas.DataFrame): DataFrame containing WhatsApp chat data with 'username' column

    Returns:
    Dict[str, str]: Mapping of original usernames to shortened names
    """
    # Count messages per user and sort by frequency (descending)
    user_counts = wa_df["username"].value_counts()

    # Create mapping with lower IDs for more frequent speakers
    user_mapping = {
        username: f"@user_{i+1}" for i, (username, _) in enumerate(user_counts.items())
    }

    return user_mapping


async def _process_conversation(conv_id: int) -> Dict:
    """Helper function to process a single conversation"""
    conv_data = df_with_conversation_ids[
        df_with_conversation_ids["conversation_id"] == conv_id
    ]
    messages = [
        Message(
            message_id=f"na-{row['date']}",
            timestamp=row["date"],
            chat_jid=group_jid,
            text=row["message"],
            sender_jid=row["username"],
            group_jid=group_jid,
        )
        for _, row in conv_data.iterrows()
    ]
    topics = await get_conversation_topics(messages)
    async with async_session() as session:
        await load_topics(
            session,
            group_jid,
            embedding_client,
            topics,
            conv_data["date"].min().to_pydatetime(),
        )
        await session.commit()


async def _bounded_process_conversation(conversation_id: int) -> Dict:
    async with semaphore:
        return await _process_conversation(conversation_id)


user_mapping = _create_user_mapping(chat_df)

# Add mapped usernames as a new column
chat_df["mapped_username"] = chat_df["username"].map(user_mapping)


# Identify conversations
df_with_conversation_ids = _identify_conversations(
    chat_df, time_threshold_minutes=60 * 3
)

conversation_ids = df_with_conversation_ids["conversation_id"].unique()
print(f"conversation_ids: {conversation_ids}")
# # If you want to limit the number of conversations to process
# conversation_ids = conversation_ids[:20]

# Create a semaphore to limit concurrency to max_concurrency tasks
max_concurrency = 30
semaphore = asyncio.Semaphore(max_concurrency)

/Users/ilan/workspace/kelet/wa_llm/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Logfire project URL: ]8;id=638978;https://logfire.pydantic.dev/ilanbenb/wa-llm\https://logfire.pydantic.dev/ilanbenb/wa-llm]8;;\

conversation_ids: [   0    1    2 ... 1080 1081 1082]


In [ ]:
# Process all conversations concurrently, limited by the semaphore
await asyncio.gather(
    *[
        _bounded_process_conversation(conversation_id)
        for conversation_id in conversation_ids
    ]
)

14:37:05.805 agent run prompt=2023-04-19 11:25:24: @user_5: 🍾
2023-04-19 11:26:39: @user_7: ..._3: לינק לקבוצת מיטאפ
https://www.meetup.com/openai-genai-tlv/
14:37:05.806   preparing model and tools run_step=1
14:37:05.806   model request
14:37:05.811 agent run prompt=2023-04-19 17:29:29: @user_1: היי, שאלה לפורום: האם מישהו מכיר...ts/tompascoe_videogames-vr-ai-ugcPost-7054408392555003904-zA28
14:37:05.812   preparing model and tools run_step=1
14:37:05.812   model request
14:37:05.815 agent run prompt=2023-04-20 16:35:24: @user_1: *מדריך להתקנת והתנעת Auto-GPT-*
...er_3: ניסית בסוף?
2023-04-20 16:40:56: @user_2: מחכה לקבל גישה
14:37:05.815   preparing model and tools run_step=1
14:37:05.815   model request
14:37:05.817 agent run prompt=2023-04-20 20:15:32: @user_2: היי חברים, הרבה הצטרפו בימים האח....linkedin.com/feed/update/urn:li:activity:7054895388008300545/
14:37:05.817   preparing model and tools run_step=1
14:37:05.817   model request
14:37:05.819 agent run prompt=2023-04-21 10:

Finished call to 'daily_ingest.daily_ingest.get_conversation_topics.<locals>._run_agent' after 15.932(s), this was the 1st time calling it.
Finished call to 'daily_ingest.daily_ingest.get_conversation_topics.<locals>._run_agent' after 22.874(s), this was the 1st time calling it.
Finished call to 'daily_ingest.daily_ingest.get_conversation_topics.<locals>._run_agent' after 23.990(s), this was the 1st time calling it.
Finished call to 'daily_ingest.daily_ingest.get_conversation_topics.<locals>._run_agent' after 18.129(s), this was the 1st time calling it.


14:37:39.001 agent run prompt=2023-05-19 04:19:35: @user_1: https://arxiv.org/pdf/2303.12712.pdf
14:37:39.003   preparing model and tools run_step=1
14:37:39.003   model request
14:37:39.260 agent run prompt=2023-05-13 03:48:17: @user_1: גם לי נתנו היום.

חושב שזו תוצאה של מיקרוסופט.
מעניין שהם מאפשרים להדליק עד 3 בו זמנית.
14:37:39.260   preparing model and tools run_step=1
14:37:39.260   model request
14:37:39.285 agent run prompt=2023-05-10 23:38:38: @user_1: בהינתן ספריה פנימית של החברה, אם...עוד איזה פקודה שתיים בפרומפט כדי לשאול חופשי ולקבל תשובה טובה.
14:37:39.285   preparing model and tools run_step=1
14:37:39.286   model request


Finished call to 'daily_ingest.daily_ingest.get_conversation_topics.<locals>._run_agent' after 18.838(s), this was the 1st time calling it.
Finished call to 'daily_ingest.daily_ingest.get_conversation_topics.<locals>._run_agent' after 15.323(s), this was the 1st time calling it.
Finished call to 'daily_ingest.daily_ingest.get_conversation_topics.<locals>._run_agent' after 17.881(s), this was the 1st time calling it.


             agent run prompt=2023-05-23 03:33:40: @user_1: ועכשיו נופל על הכנס של שעשוע ועל aws :)
14:37:39.500   handle model response
14:37:39.519 agent run prompt=2023-05-16 22:59:37: @user_6: https://github.com/microsoft/gui...://twitter.com/untitled01ipynb/status/1658588955044982784?s=20
14:37:39.520   preparing model and tools run_step=1
14:37:39.521   model request


...

14:58:24.738 agent run prompt=[Scrubbed due to 'auth']
14:58:24.742   preparing model and tools run_step=1
14:58:24.742   model request
             agent run prompt=2025-02-10 14:26:56: @user_3: היי, יש בנצ׳מרק עדכני עבור o3 בה...אמצעות דברים שבדרך כלל נכללים תחת mechanistic interpretability
14:58:25.051   handle model response


Finished call to 'daily_ingest.daily_ingest.get_conversation_topics.<locals>._run_agent' after 58.521(s), this was the 5th time calling it.
Finished call to 'daily_ingest.daily_ingest.get_conversation_topics.<locals>._run_agent' after 40.636(s), this was the 3rd time calling it.


             agent run prompt=2025-02-13 09:05:42: @user_2: למישהו יש במקרה הרצאת מבוא לGenA...@user_1: לדעתי זה Big deal לסוכנים אוטונומיים שמפתחים פרונטאנד
14:58:27.599   handle model response
             agent run prompt=2025-02-07 13:41:25: @user_1: Ilan Benborhoum added +972 54‐251‐6456
14:58:28.284   handle model response
             agent run prompt=2025-02-09 09:01:37: @user_1: https://x.com/AmitMandelAI/statu...50:58: @user_4: טעות במוצר
2025-02-09 11:51:01: @user_4: מתנצל
14:58:28.891   handle model response


Finished call to 'daily_ingest.daily_ingest.get_conversation_topics.<locals>._run_agent' after 88.595(s), this was the 5th time calling it.


             agent run prompt=2025-02-16 07:06:11: @user_3: אנטרופיק די מובילים בזה.
ה citat... בצדק :) וזה אכן נכשל בגדול בתסריטים עם רמת קושי גבוהה/בינונית
14:58:29.119   handle model response
             agent run prompt=2025-02-16 00:20:09: @user_1: זה בדיוק העניין, אנחנו בעצמנו לא...הסכמה גבוהה עם מקור מסוים, לעומת מצב שבו המודל "מתאמץ" להמציא.
14:58:29.747   handle model response
14:58:30.109 agent run prompt=2025-02-12 14:17:04: @user_1: כרגע הזנת לאופן איי, תוכנית פעול...@user_5: גם ה 4090 היה ככה בשבועות הראשונים וגם 3090. יעבור...
14:58:30.113   preparing model and tools run_step=1
14:58:30.113   model request
             agent run prompt=2025-02-11 08:55:51: @user_9: נראה לי הדרך הכי טובה למנוע מהמו...מדי, עם זה אין את הבעיות הללו ולכן קל יותר לייצר כתוביות למשל.
14:58:30.953   handle model response
14:58:34.251 agent run prompt=2025-01-29 09:55:30: @user_21: למי שמעוניינים, השקנו לידרבורד ...16:56: @user_6: אחלה פורמט!
למה אבל המבחני איות האלה מעניינים?
14:58:34.255   pre